In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
from transformers import AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/data/mistral/query-to-mql/exp-10/nov-20/merged-model")
model.to_bettertransformer()

In [4]:
!sudo pip install -q git+https://github.com/huggingface/optimum.git

In [10]:
!sudo pip install onnxruntime

     |████████████████████████████████| 6.4MB 7.2MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 105.4MB/s eta 0:00:01
  Found existing installation: numpy 1.21.1
    Uninstalling numpy-1.21.1:
      Successfully uninstalled numpy-1.21.1


In [12]:
from optimum.pipelines import pipeline

In [11]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForCausalLM
import torch

In [ ]:


model_path = "path/to/gpt-something.onnx"
# the model_path also needs to include the `config.json` from the original model
tokenizer_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = ORTModelForCausalLM.from_pretrained(model_path)

inputs = tokenizer("My name is Philipp and I live in Germany.", return_tensors="pt")

gen_tokens = model.generate(**inputs,do_sample=True,temperature=0.9, min_length=20,max_length=20)
tokenizer.batch_decode(gen_tokens)

In [13]:
onnx = pipeline("question-answering", model="/data/mistral/query-to-mql/exp-10/nov-20/merged-model", accelerator="ort")

ValueError: Task question-answering is not supported for the ONNX Runtime pipeline. Supported tasks are []

In [ ]:
question = "What's my name?"
context = "My name is Philipp and I live in Nuremberg."

pred = onnx_qa(question=question, context=context)